In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
drivers_df = pd.read_csv('F1KaggleData/drivers.csv')
races_df = pd.read_csv('F1KaggleData/races.csv')
circuits_df = pd.read_csv('F1KaggleData/circuits.csv')
results_df = pd.read_csv('F1KaggleData/results.csv')


In [3]:
display(drivers_df.head(5))
display(races_df.head(5))
display(circuits_df.head(5))
display(results_df.head(5))


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,NaN,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,NaN,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111,NaN,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,NaN,http://en.wikipedia.org/wiki/Istanbul_Park


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1


In [4]:
display('Total Number of Circuits used in F1: {}'.format(circuits_df['circuitId'].max()))
display('Total Drivers in F1: {}'.format(drivers_df['driverId'].max()))


'Total Number of Circuits used in F1: 73'

'Total Drivers in F1: 843'

Drivers with a win?  

How do we work this out... 

In [5]:
wins_df = results_df.loc[results_df['position'] == 1.0]

display(wins_df.sample(10))

drivers_with_wins = wins_df['driverId'].unique()

display(drivers_with_wins)

display('{} drivers in F1 have won a race'.format(len(drivers_with_wins)))

names_of_drivers_with_wins = drivers_df[['driverRef', 'nationality']].loc[drivers_df['driverId'].isin(drivers_with_wins)]

display(names_of_drivers_with_wins)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
18524,18525,767,475,118,1.0,2,1.0,1,1,9.0,75,04:49.2,7489200.0,NaN,NaN,NaN,NaN,1
13195,13196,541,199,6,11.0,2,1.0,1,1,9.0,59,40:48.8,6048800.0,NaN,NaN,NaN,NaN,1
20728,20731,354,20,9,5.0,2,1.0,1,1,25.0,71,33:11.8,5591803.0,70.0,6.0,01:14.3,208.828,1
19767,19768,822,647,6,101.0,1,1.0,1,1,9.0,18,06:13.3,11173300.0,NaN,NaN,NaN,NaN,1
4490,4491,229,44,27,9.0,14,1.0,1,1,10.0,75,00:45.6,7245629.0,NaN,NaN,NaN,NaN,1
2842,2843,154,30,6,1.0,3,1.0,1,1,10.0,36,08:05.0,4085002.0,NaN,NaN,NaN,NaN,1
10466,10467,445,182,1,8.0,3,1.0,1,1,9.0,71,29:28.5,5368532.0,NaN,NaN,NaN,NaN,1
5275,5276,262,30,22,5.0,1,1.0,1,1,10.0,69,44:32.1,6272100.0,NaN,NaN,NaN,NaN,1
16983,16984,699,373,172,5.0,1,1.0,1,1,9.0,85,06:46.0,7606000.0,NaN,NaN,NaN,NaN,1
13316,13317,546,207,32,5.0,2,1.0,1,1,9.0,80,51:35.5,6695470.0,NaN,NaN,NaN,NaN,1


array([  1,   8,  13,   9,   5,  20,   4,  21,  30,  18,  31,  15,  22,
        14,  23,  57,  56,  49,  65,  71,  35,  77,  44,  55,  95, 117,
       102, 119, 137, 123,  17, 145, 173, 105, 177, 182, 187, 175, 163,
       202, 178, 199, 203, 172, 219, 221, 222, 223, 207, 238, 255, 231,
       224, 250, 200, 230, 304, 328, 309, 306, 235, 346, 320, 327, 356,
       358, 345, 289, 373, 360, 364, 341, 375, 386, 385, 475, 476, 403,
       394, 404, 509, 449, 347, 479, 427, 525, 578, 581, 579, 611, 577,
       559, 628, 657, 498, 647, 642, 641, 526, 766, 786, 593,   3, 813,
       817, 830, 822], dtype=int64)

'107 drivers in F1 have won a race'

,driverRef,nationality
0,hamilton,British
2,rosberg,German
3,alonso,Spanish
4,kovalainen,Finnish
7,raikkonen,Finnish
8,kubica,Polish
12,massa,Brazilian
13,coulthard,British
14,trulli,Italian
16,webber,Australian


In [6]:
# can we get a pie chart of the nationality breakdown

winners_by_nationality = names_of_drivers_with_wins['nationality'].value_counts()

display(winners_by_nationality)

names_of_drivers_with_wins['nationality'].value_counts().plot.pie(subplots=True)




British          19
American         15
Italian          15
French           12
German            7
Brazilian         6
Finnish           5
Australian        4
Austrian          3
Argentine         3
Swedish           3
Belgian           2
Canadian          2
New Zealander     2
Swiss             2
Venezuelan        1
Mexican           1
South African     1
Polish            1
Dutch             1
Colombian         1
Spanish           1
Name: nationality, dtype: int64

array([<matplotlib.axes._subplots.AxesSubplot object at 0x00000209DB313400>],
      dtype=object)

In [7]:
# ok, lets try winners by decade here. 

# however lets do this via some dictionary comprehension 

# pandas first
display(results_df.head(5))
# display(races_df.head(5))

first_race = races_df['date'].min()
display('Date of First Race in dataset: {}'.format(first_race))
last_race = races_df['date'].max()
display('Date of Last Race in dataset: {}'.format(last_race))

# sort it becuase this current format hurts my brain 
races_df.sort_values(by=['date'], inplace=True)
display(races_df.head(5))
list_of_dates = races_df['date'].values
# display(list_of_dates)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
1,2,18,2,2,3.0,5,2.0,2,2,8.0,58,5.478,5696094.0,41.0,3.0,01:27.7,217.586,1
2,3,18,3,3,7.0,7,3.0,3,3,6.0,58,8.163,5698779.0,41.0,5.0,01:28.1,216.719,1
3,4,18,4,4,5.0,11,4.0,4,4,5.0,58,17.181,5707797.0,58.0,7.0,01:28.6,215.464,1
4,5,18,5,1,23.0,3,5.0,5,5,4.0,58,18.014,5708630.0,43.0,1.0,01:27.4,218.385,1


'Date of First Race in dataset: 1950-05-13'

'Date of Last Race in dataset: 2018-11-25'

,raceId,year,round,circuitId,name,date,time,url
832,833,1950,1,9,British Grand Prix,1950-05-13,,http://en.wikipedia.org/wiki/1950_British_Gran...
833,834,1950,2,6,Monaco Grand Prix,1950-05-21,,http://en.wikipedia.org/wiki/1950_Monaco_Grand...
834,835,1950,3,19,Indianapolis 500,1950-05-30,,http://en.wikipedia.org/wiki/1950_Indianapolis...
835,836,1950,4,66,Swiss Grand Prix,1950-06-04,,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...
836,837,1950,5,13,Belgian Grand Prix,1950-06-18,,http://en.wikipedia.org/wiki/1950_Belgian_Gran...


In [8]:
# display(results_df.head(10))

winner_of_each_race_df = results_df.loc[results_df['position'] == 1.0]
winners = set(winner_of_each_race_df['driverId'].to_list())
display(winner_of_each_race_df.head(5))
count_of_wins_per_driver = winner_of_each_race_df['driverId'].value_counts()
display(count_of_wins_per_driver)
display(winners)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22.0,1,1.0,1,1,10.0,58,34:50.6,5690616.0,39.0,2.0,01:27.5,218.3,1
22,23,19,8,6,1.0,2,1.0,1,1,10.0,56,31:18.6,5478555.0,37.0,2.0,01:35.4,209.158,1
44,45,20,13,6,2.0,2,1.0,1,1,10.0,57,31:07.0,5466970.0,38.0,3.0,01:33.6,208.153,1
66,67,21,8,6,1.0,1,1.0,1,1,10.0,66,38:19.1,5899051.0,46.0,1.0,01:21.7,205.191,1
88,89,22,13,6,2.0,1,1.0,1,1,10.0,58,26:49.5,5209451.0,16.0,3.0,01:26.7,221.734,1


30     91
1      62
117    51
20     47
102    41
4      32
95     31
328    27
182    25
373    25
579    24
3      23
137    23
71     22
8      20
57     20
475    16
18     15
224    14
289    14
356    14
647    13
14     13
178    12
199    12
207    12
35     11
22     11
13     11
222    10
       ..
628     1
386     1
641     1
449     1
15      1
766     1
611     1
9       1
786     1
5       1
813     1
385     1
327     1
320     1
509     1
593     1
44      1
200     1
55      1
306     1
394     1
255     1
250     1
577     1
559     1
526     1
230     1
525     1
404     1
375     1
Name: driverId, Length: 107, dtype: int64

{1,
 3,
 4,
 5,
 8,
 9,
 13,
 14,
 15,
 17,
 18,
 20,
 21,
 22,
 23,
 30,
 31,
 35,
 44,
 49,
 55,
 56,
 57,
 65,
 71,
 77,
 95,
 102,
 105,
 117,
 119,
 123,
 137,
 145,
 163,
 172,
 173,
 175,
 177,
 178,
 182,
 187,
 199,
 200,
 202,
 203,
 207,
 219,
 221,
 222,
 223,
 224,
 230,
 231,
 235,
 238,
 250,
 255,
 289,
 304,
 306,
 309,
 320,
 327,
 328,
 341,
 345,
 346,
 347,
 356,
 358,
 360,
 364,
 373,
 375,
 385,
 386,
 394,
 403,
 404,
 427,
 449,
 475,
 476,
 479,
 498,
 509,
 525,
 526,
 559,
 577,
 578,
 579,
 581,
 593,
 611,
 628,
 641,
 642,
 647,
 657,
 766,
 786,
 813,
 817,
 822,
 830}

In [9]:
# now lets join the data from the results df and the drivers_df 

winner_code = drivers_df.loc[drivers_df['driverId'].isin(winner_of_each_race_df['driverId'].unique())][['driverId','driverRef']]

# display(winner_of_each_race_df['driverId'].unique())
display(winner_code)

,driverId,driverRef
0,1,hamilton
2,3,rosberg
3,4,alonso
4,5,kovalainen
7,8,raikkonen
8,9,kubica
12,13,massa
13,14,coulthard
14,15,trulli
16,17,webber


In [10]:
comb = pd.merge(winner_code, winner_of_each_race_df)

comb2 = pd.merge(winner_code, winner_of_each_race_df, on='driverId')

#display(comb2)
x = comb.driverRef.value_counts()

#display(x)
#display(type(x))

In [13]:
# Select to see wins by driver drop down 

import ipywidgets as widgets
from IPython.display import clear_output

def f1_winner_dropdown_eventhandler(change):

    with out:
        clear_output()
        selected_driver_id = int(winner_code.loc[winner_code['driverRef'] == change.new]['driverId'])
        display(selected_driver_id)
        display(results_df.loc[(results_df['driverId']== selected_driver_id) & (results_df['position'] == 1.0)])


f1_winner_dropdown = widgets.Dropdown(
                                    options=winner_code['driverRef'].to_list(),
                                    value=None,
                                    description='Select F1 Winner: ',
                                    disabled=False,
                                    )

# now lets restrict the results purely to the user selected driver.... 
display(f1_winner_dropdown)
out = widgets.Output()
display(out)

f1_winner_dropdown.observe(f1_winner_dropdown_eventhandler, names='value')

Dropdown(description='Select F1 Winner: ', options=('hamilton', 'rosberg', 'alonso', 'kovalainen', 'raikkonen'…

Output()

In [12]:
# now I want to see the number of different winners per decade 

# display(drivers_df.head(5))
# display(races_df.head(5))
# display(circuits_df.head(5))
# display(results_df.head(5))

# ok, lets take the races and then see if we can do something with the series. 
races_df['datetime'] = pd.to_datetime(races_df['date'])
races_df.set_index('datetime', inplace=True)
races_df.sort_index(axis = 0, inplace=True) 
# isplay(races_df.head(5))

# now we need to go and find the winner for each one and add it.

cut_down_results = results_df[['raceId', 'driverId']].loc[results_df['positionText'] == '1']
# display(cut_down_results.head(5))

answer_df = pd.merge(races_df, cut_down_results, on='raceId')
answer_df = pd.merge(answer_df, drivers_df[['driverId', 'driverRef']], on='driverId')
answer_df.sort_values(by='date', inplace=True, ascending=True)
display(answer_df.head(10))
display(drivers_df.head(5))



,raceId,year,round,circuitId,name,date,time,url,driverId,driverRef
0,833,1950,1,9,British Grand Prix,1950-05-13,,http://en.wikipedia.org/wiki/1950_British_Gran...,642,farina
5,834,1950,2,6,Monaco Grand Prix,1950-05-21,,http://en.wikipedia.org/wiki/1950_Monaco_Grand...,579,fangio
29,835,1950,3,19,Indianapolis 500,1950-05-30,,http://en.wikipedia.org/wiki/1950_Indianapolis...,593,parsons
1,836,1950,4,66,Swiss Grand Prix,1950-06-04,,http://en.wikipedia.org/wiki/1950_Swiss_Grand_...,642,farina
6,837,1950,5,13,Belgian Grand Prix,1950-06-18,,http://en.wikipedia.org/wiki/1950_Belgian_Gran...,579,fangio
7,838,1950,6,55,French Grand Prix,1950-07-02,,http://en.wikipedia.org/wiki/1950_French_Grand...,579,fangio
2,839,1950,7,14,Italian Grand Prix,1950-09-03,,http://en.wikipedia.org/wiki/1950_Italian_Gran...,642,farina
8,825,1951,1,66,Swiss Grand Prix,1951-05-27,,http://en.wikipedia.org/wiki/1951_Swiss_Grand_...,579,fangio
30,826,1951,2,19,Indianapolis 500,1951-05-30,,http://en.wikipedia.org/wiki/1951_Indianapolis...,766,wallard
3,827,1951,3,13,Belgian Grand Prix,1951-06-17,,http://en.wikipedia.org/wiki/1951_Belgian_Gran...,642,farina


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
